In [6]:
from fenics import *
import sys
sys.path.append('../data/')
sys.path.append('../graphnics/')
sys.path.append('../applications/')
sys.path.append('../../NetworkGen/')
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import imp
from graphnics import *

# Coupled 1d-3d demo

`graphnics` (in development) is a python library that combines `networkx` and `FEniCS`. The module provides functionality for meshing and assembling variational forms on network structures. By combining it with `fenics_ii` we can further use it to solve coupled 1D-3D flow problems.

## Background

Coupled 1d-3d flow models are used to model e.g. 
- blood flow in vascularized tissue,
- water flow through the root network in soil,
- fluid flow through wells drilled in a reservoir.
- 
Each of these application share a common geometry: A network of flow channels $\Lambda$ (blood vessels, roots or wells) embedded in a surrounding porous media $\Omega \subset \mathbb{R}^3$ (tissue, soil or reservoir).

## Model equations

The [coupled 1d-3d flow model](https://www.esaim-m2an.org/articles/m2an/abs/2019/06/m2an180210/m2an180210.html) relates e.g. a fluid pressure $\hat{u}$ in a network $\Lambda$ with the fluid pressure $u$ in the surrounding domain $\Omega$:

$$
\begin{align*}
- \Delta u &= \beta(\hat{u}-\bar{u})\delta_\Gamma \quad &&\text{ in } \Omega, \\
- \partial_{ss} \hat{u} &= -\beta(\hat{u}-\bar{u}) \quad &&\text{ on } \Lambda
\end{align*}
$$

Here, $\delta_\Lambda$ is the Dirac line source 
$$
\begin{align*}
\int_\Omega \phi \delta_\Lambda \, \mathrm{d}x &= \int_\Omega \phi(s) \, \mathrm{d}s  \\
\end{align*}
$$
for $\phi \in C^0(\Omega)$ and $\bar{u}$ denotes the 3d pressure *averaged* the lateral boundary of the flow channel:
$$
\begin{align*}
\bar{u}(s) &= \int_{\partial C(s)} u  \, \mathrm{d}\theta. \\
\end{align*}
$$

The parameter $\beta$ adjusts the permeability of the "membrane" that separates the flow domains $\Lambda$ and $\Omega$. Increasing $\beta$ will lead to an increase in the fluid exchange between the two.

## Implementation

### Making the domains

We consider a Y-shaped network embedded inside a box:

In [7]:
# Make 1d y bifurcation
G = make_Y_bifurcation(dim=3)
G.make_mesh(n=4)
mesh1d = G.global_mesh

pos = nx.get_node_attributes(G, "pos")
node_coords = np.asarray(list(pos.values()))

# fit 3d box around it 
mesh3d = UnitCubeMesh(40, 40, 40)

c = mesh3d.coordinates()

xmin, ymin, zmin = np.min(node_coords, axis=0)
xmax, ymax, zmax = np.max(node_coords, axis=0)

c[:, 0] *= (xmax - xmin) * 1.2
c[:, 1] *= (ymax - ymin) * 1.2
c[:, 2] *= 0.5
c[:, 2] -= 0.25
c[:, 0] -= np.abs(xmin)*1.2
c[:, 1] -= 0.1

File("plots/coupled1d3d/box.pvd") << Function(FunctionSpace(mesh3d, "CG", 1))
File("plots/coupled1d3d/network.pvd") << Function(FunctionSpace(mesh1d, "CG", 1))

### Parameters

We assign a given pressure drop across the bifurcation network, and a zero pressure on the boundaries of the box:

In [8]:
# Pressure boundary conditions
u_bc_3 = Constant(0)
u_bc_1 = Expression('x[1]', degree=2)

# Permeability of "membrane" separating $\Omega$ and $\Lambda$
beta = Constant(1)

### Variational formulation

The variational problem we want to solve reads:

Find $u \in H^1(\Omega)$ and $\hat{u} \in H^1(\Lambda)$ such that
$$
\begin{align*}
(\nabla u, \nabla v)_\Omega &= \vert \partial C(s)\vert (\beta(\hat{u}-\bar{u}), \bar{v})_\Lambda, \\
(\partial_{s} \hat{u}, \partial_{s} \hat{v})_\Lambda &= -(\beta(\hat{u}-\bar{u}), \hat{v})_\Lambda
\end{align*}
$$
for all $v \in H^1(\Omega)$ and $\hat{v} \in H^1(\Lambda)$.

Here $\vert \partial C(s)\vert$ denotes the circumference of the channel cross-section; for pipes with radius $R$ we have $\vert \partial C(s)\vert=2\pi R$.


### Discretization

We discretize using linear elements, setting $u_h = \sum_{i=1}^n u_i \phi_i$ and $\hat{u}_h = \sum_{i=1}^{\hat{n}} \hat{u}_i \hat{\phi}_i$ where $\phi_i$ and $\hat{\phi}_i$ are the 3d and 1d hat functions, respectively.

In block matrix form our problem then reads:

$$
\begin{align}
\begin{pmatrix}
(\nabla \phi_j, \nabla \phi_i)_\Omega + \vert \partial C(s)\vert (\bar{\phi}_j, \bar{\phi}_i)_\Lambda& \quad -\vert \partial C(s)\vert \beta (\hat{\phi}_j, \bar{\phi}_i)_\Lambda \\
 - \beta (\bar{\phi}_j, \hat{\phi}_i)_\Lambda & \quad (\partial_{s} \hat{\phi}_j, \partial_{s} \hat{\phi}_i)_\Lambda +(\beta \hat{\phi}_j, \hat{\phi}_i)_\Lambda
\end{pmatrix}
\begin{pmatrix}
u_h \\
\hat{u}_h
\end{pmatrix}
= 
\begin{pmatrix}
0 \\ 0
\end{pmatrix} 
\end{align}
$$

We construct the forms used in each block as follows:

In [10]:
# Pressure space on global mesh
V3 = FunctionSpace(mesh3d, "CG", 1)
V1 = FunctionSpace(mesh1d, "CG", 1)
W = [V3, V1]

u3, u1 = list(map(TrialFunction, W))
v3, v1 = list(map(TestFunction, W))

# Averaging surface
cylinder = Circle(radius=0.1, degree=10)
circum = Constant(2*3.14)  #circumference

Pi_u = Average(u3, mesh1d, cylinder)
Pi_v = Average(v3, mesh1d, cylinder)

dxGamma = Measure("dx", domain=mesh1d)

# blocks
a00 = inner(grad(u3), grad(v3)) * dx + circum* beta * inner(Pi_u, Pi_v) * dxGamma
a01 = -beta * circum * inner(u1, Pi_v) * dxGamma
a10 = -beta * inner(Pi_u, v1) * dxGamma
a11 = inner(grad(u1), grad(v1)) * dx + beta * inner(u1, v1) * dxGamma

# right-hand side
L0 = inner(Constant(0), Pi_v) * dxGamma
L1 = inner(Constant(0), v1) * dxGamma

### Assembly

Now we just need to make the nested list representing the block matrix and ask `fenics_ii` to solve our system:

In [11]:

a = [[a00, a01], [a10, a11]]
L = [L0, L1]

W_bcs = [
    [DirichletBC(V3, u_bc_3, "on_boundary")],
    [DirichletBC(V1, u_bc_1, "on_boundary")],
]

A, b = map(ii_assemble, (a, L))
A, b = apply_bc(A, b, W_bcs)
A, b = map(ii_convert, (A, b))

wh = ii_Function(W)
solver = LUSolver(A, "mumps")
solver.solve(wh.vector(), b)

File('plots/coupled1d3d/pressure1d.pvd') << wh[1]
File('plots/coupled1d3d/pressure3d.pvd') << wh[0]

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


### Result

Finally, we can plot the results using paraview. If you open the files `pressure1d.pvd` and `pressure3d.pvd` located in the `plots/coupled1d3d/` folder you should see something like this:

<img alt="alt_text" width="500px" src="coupled1d3d.png" />